In [1]:
import os
import pandas as pd
import json

os.chdir(r'T:\laupodteam\AIOS\Bram\language_modeling\MEDICAL_TEXT\RAW\NtvG')

In [2]:
nvtg = pd.read_feather("./ntvg_articles.feather")

In [3]:
# make function to pull out a list of dictionaries with the following keys:
# - body_clean
# - summary_clean
# - category
# - domain

# parameters:
# - stratified
# - fraction
# - random_state

def pull_dictionaries(df: pd.DataFrame, 
                      stratified: bool,
                      fraction: float, 
                      random_state: int, 
                      exclude_dict: dict=None):
    
    if isinstance(exclude_dict, dict):
        for k,v in exclude_dict.items():
            for _v in v:
                df = df.loc[df[k]!=_v]
    
    if stratified:
        # TODO: implement stratified sampling
        df = df.sample(frac=fraction, random_state=random_state)
    else:
        df = df.sample(frac=fraction, random_state=random_state)
    df = df.reset_index(drop=True)
    dictionaries = []
    for i in range(len(df)):
        summaries = df.loc[i, "summary_clean"]
        explanation = df.loc[i, "explanation_clean"]
        information_framework = df.loc[i, "information_framework_clean"]
        intro = df.loc[i, "intro_clean"]
        
        summary_string = "\r\n".join(summaries) if summaries is not None else None
        explanation_string = "\r\n".join(explanation) if explanation is not None else None
        information_framework_string = "\r\n".join(information_framework) if information_framework is not None else None
        intro_string = "\r\n".join(intro) if intro is not None else None
        
        txts = "\r\n".join(df.loc[i, "body_clean"])
        
        id = df.loc[i, "id"]        
        dictionary = {
                    "id": id,
                    "body_clean": txts,
                    "summary_clean": summary_string,
                    'explanation_clean': explanation_string,
                    'information_framework_clean': information_framework_string,
                    'intro_clean': intro_string,
                    "category": df.loc[i, "category"],
                    "domain": df.loc[i, "domain"]
                }
        dictionaries.append(dictionary)
    return dictionaries

In [21]:
dicts = pull_dictionaries(nvtg, stratified=True, fraction=1., random_state=42, 
                          exclude_dict={'category':['Redactioneel'], 'domain': []})

extraction_df = pd.DataFrame(dicts)

extraction_df.body_clean = extraction_df.body_clean.str.replace(r'\[{2}.*\]{2}', '', regex=True)
extraction_df.summary_clean = extraction_df.summary_clean.str.replace(r'\[{2}.*\]{2}', '', regex=True)
extraction_df.explanation_clean = extraction_df.explanation_clean.str.replace(r'\[{2}.*\]{2}', '', regex=True)
extraction_df.information_framework_clean = extraction_df.information_framework_clean.str.replace(r'\[{2}.*\]{2}', '', regex=True)
extraction_df.intro_clean = extraction_df.intro_clean.str.replace(r'\[{2}.*\]{2}', '', regex=True)

In [25]:
extraction_df.sample(frac=0.01).to_json('./samples.jsonl', orient='records', lines=True, force_ascii=False)
extraction_df.to_json('./ntvg.jsonl', orient='records', lines=True, force_ascii=False)

In [28]:
extraction_df[extraction_df.summary_clean.notna()]

,id,body_clean,summary_clean,explanation_clean,information_framework_clean,intro_clean,category,domain
1,1269419,Lymeziekte kent een grote variatie in klinisch...,In Europa komen steeds vaker tekenbeetziekten ...,None,None,None,Casuïstiek,Hart en vaten
6,1215499,Het komt helaas nogal eens voor dat een juiste...,Een 19-jarige Turkse vrouw had langdurig besta...,None,None,None,Casuïstiek,Interne vakken
8,1216168,"Spondylitis ankylopoetica, ook wel ziekte van ...",Kosteneffectiviteitsanalyse van begeleide groe...,None,None,None,Onderzoek,Bewegingsapparaat
10,1213213,In de verpleeghuizen Mariënhaven (psychogeria...,Vaststellen of verhoogde alertheid op longembo...,None,None,None,Onderzoek,Longziekten
18,1238199,Acetylsalicylzuur (ASA) wordt veel voorgeschre...,,None,None,None,In het kort,Maag-darm-leverziekten
...,...,...,...,...,...,...,...,...
41613,1240362,Sinds 2003 wordt er binnen de verloskunde in N...,Het verkrijgen van inzicht in de kosten en gez...,None,None,None,Onderzoek,Zorgorganisatie
41618,1213024,Sedert het einde van de jaren tachtig stijgt h...,Bij twee vrouwen van 71 en 59 jaar en een man ...,None,None,None,Casuïstiek,Interne vakken
41619,1223286,De huidige behandeling van patiënten met een ...,Vaststellen van therapietrouw bij Nederlandse ...,None,None,None,Onderzoek,Geneesmiddelen
41620,1270598,De aanwezigheid van verworven klonale genetisc...,,None,None,None,Stand van zaken,Genetica
